# Testing Quantized Model

In [1]:
import os
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
print('TensorFlow Version: %s' % tf.__version__)

TensorFlow Version: 1.12.0


In [4]:
model_path = os.path.join('_quantized', 'quantize_eval_model.pb')

if not os.path.exists(model_path):
    print('Quantized model %s cannot be found...' % model_path)

In [5]:
# Need to run this before importing quantized graph
tf.contrib.resampler

<module 'tensorflow.contrib.resampler' from '/home/pentaxmedical/program/miniconda3/envs/vitis-ai-tensorflow/lib/python3.6/site-packages/tensorflow/contrib/resampler/__init__.py'>

## Load model

In [6]:
graph_def = tf.GraphDef()
graph_def.ParseFromString(tf.gfile.GFile(model_path, "rb").read())

4804757

In [7]:
graph = tf.Graph()
graph.as_default()

In [8]:
tf.import_graph_def(graph_def, name = "")

In [9]:
# for node in graph_def.node:
#     print('%s' % (node.name) )

In [10]:
input_tensor = tf.get_default_graph().get_tensor_by_name('conv2d_input:0')
output_tensor = tf.get_default_graph().get_tensor_by_name('dense_1/Softmax:0')

## Load dataset

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train/255
x_test  = x_test/255

## Evaluate

- Use tf.Session

In [12]:
# for i in range(0, iter):
batch_size = 100
iter = int(len(x_train)/batch_size)

result = np.zeros(len(x_train))

with tf.Session() as sess:
    for i in range(0, iter):            
        img = np.expand_dims(x_train[batch_size*i:batch_size*(i+1)], axis=3)

        feed_dict = {input_tensor: img}
        out_0 = sess.run([output_tensor], feed_dict)

        result[batch_size*i:batch_size*(i+1)] = np.argmax(out_0[0], axis=1)

In [13]:
print('Accuracy: %d / %d' % (np.sum(result == y_train), len(x_train)) )

Accuracy: 59892 / 60000
